In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import simfin as sf
from simfin.names import *
from datetime import datetime

In [2]:
load_dotenv()

api_key = os.getenv("API_KEY")

print(api_key[:5])  


9104e


In [3]:

# Set SimFin data directory
sf.set_api_key(api_key)

sf.set_data_dir('/Users/ayushsingh/Desktop/MBD/Python 2/Group Assignment/simfin_data/')

# Load datasets
df_companies = sf.load_companies(market='us')
df_share_prices = sf.load_shareprices(market='us', variant='daily')


Dataset "us-companies" on disk (6 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (6 days old).
- Loading from disk ... 

/opt/anaconda3/lib/python3.12/site-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,
/opt/anaconda3/lib/python3.12/site-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!


In [4]:
# List of dataframes
dfs = [df_companies, df_share_prices]

# Process each dataframe
for df in dfs:
    # Reset index
    df.reset_index(inplace=True)

    # Convert all column names to lowercase
    df.columns = df.columns.str.lower()

    # Convert 'date' columns to datetime if present
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")

print("✅ All datasets processed successfully!")


✅ All datasets processed successfully!


In [5]:
current_year = datetime.now().year
three_years_ago = current_year - 3

# Define the tickers to keep
selected_tickers = ["AAPL", "WMT", "GOOG", "TSLA", "NVDA"]

# Filter for only the selected bank tickers
df_share_prices = df_share_prices[df_share_prices["ticker"].isin(selected_tickers)]

# Keep only data from the last 3 years
df_share_prices = df_share_prices[df_share_prices["date"].dt.year >= three_years_ago]

# Save filtered data
df_share_prices.to_csv("share_prices_last_3_years.csv", index=False)

print("✅ Share price data (last 3 years) saved as 'share_prices_banks_last_3_years.csv'.")


✅ Share price data (last 3 years) saved as 'share_prices_banks_last_3_years.csv'.


In [6]:
df_share_prices.to_csv("df_final.csv", index=False)

In [7]:
df = df_share_prices

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['ticker', 'date'])

# Remove unnecessary columns
df = df[['ticker', 'date', 'open', 'high', 'low', 'close', 'adj. close', 'volume']]

# Feature Engineering
# Define trend: 1 = Up, 0 = Down (based on next day's closing price)
df['trend'] = (df.groupby('ticker')['adj. close'].shift(-1) > df['adj. close']).astype(int)
df['daily_return'] = df.groupby('ticker')['adj. close'].pct_change()
df['volatility'] = df.groupby('ticker')['daily_return'].rolling(window=5).std().reset_index(level=0, drop=True)
df['5_day_ma'] = df.groupby('ticker')['adj. close'].rolling(window=5).mean().reset_index(level=0, drop=True)
df['10_day_ma'] = df.groupby('ticker')['adj. close'].rolling(window=10).mean().reset_index(level=0, drop=True)
df['20_day_ma'] = df.groupby('ticker')['adj. close'].rolling(window=20).mean().reset_index(level=0, drop=True)

# Shift close price to serve as next day's prediction target
df['next_day_close'] = df.groupby('ticker')['adj. close'].shift(-1)

# Drop rows with NaN values (caused by rolling operations)
df = df.dropna()

# Normalize numeric features
'''scaler = MinMaxScaler()
numerical_cols = ['open', 'high', 'low', 'close', 'adj. close', 'volume', 'daily_return', 'volatility', '5_day_ma', '10_day_ma', '20_day_ma']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])'''

# Save processed data
df.to_csv("processed_stock_data.csv", index=False)


In [9]:
df

,ticker,date,open,high,low,close,adj. close,volume,trend,daily_return,volatility,5_day_ma,10_day_ma,20_day_ma,next_day_close
14981,AAPL,2022-01-31,170.16,175.00,169.51,174.78,171.78,115541590,0,0.026104,0.033082,161.934,162.009,166.9460,171.61
14982,AAPL,2022-02-01,174.01,174.84,172.31,174.61,171.61,86213911,1,-0.000990,0.031186,164.848,162.482,166.5820,172.82
14983,AAPL,2022-02-02,174.75,175.88,173.33,175.84,172.82,84914256,0,0.007051,0.030205,168.022,163.426,166.3925,169.93
14984,AAPL,2022-02-03,174.48,176.24,172.12,172.90,169.93,89418074,0,-0.016723,0.033274,170.710,164.250,166.2930,169.65
14985,AAPL,2022-02-04,171.68,174.10,170.68,172.39,169.65,82465400,0,-0.001648,0.015624,171.158,165.253,166.3230,168.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5654407,WMT,2024-03-07,61.00,61.07,60.34,60.36,59.65,16195303,0,-0.003508,0.006366,59.102,58.797,57.5850,59.41
5654408,WMT,2024-03-08,60.00,60.28,59.73,60.12,59.41,12143645,1,-0.004023,0.007764,59.370,58.955,57.7660,59.95
5654409,WMT,2024-03-11,60.10,60.71,59.87,60.66,59.95,14062631,1,0.009089,0.007758,59.640,59.060,57.9755,60.69
5654410,WMT,2024-03-12,60.95,61.56,60.65,61.41,60.69,16555854,0,0.012344,0.007730,59.912,59.240,58.2050,60.56
